**Back to the Future**

by Annika Dahlmann, Cameron Davis, Kyle O'Laughlin, and Nathan Tseng

In [ ]:
import os
import time
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision
from sklearn.metrics import average_precision_score as ap_score
from torch.utils.data import DataLoader,Subset
from torchvision import datasets, models, transforms
from torch.utils.data.dataset import Dataset
from tqdm import tqdm
import h5py
from skimage import color
from torchsummary import summary
import argparse
import torch.utils.data as data
from skimage.metrics import peak_signal_noise_ratio
from skimage.measure import compare_ssim

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
if torch.cuda.is_available():
    print("Using the GPU. You are good to go!")
    device = torch.device('cuda:0')
else:
    raise Exception("WARNING: Could not find GPU! Using CPU only. \
To enable GPU, please to go Edit > Notebook Settings > Hardware \
Accelerator and select GPU.")

Using the GPU. You are good to go!


In [ ]:
IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
    '.tif', '.TIF', '.tiff', '.TIFF',
]


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)


def make_dataset(dir, max_dataset_size=float("inf")):
    images = []
    assert os.path.isdir(dir) or os.path.islink(dir), '%s is not a valid directory' % dir

    for root, _, fnames in sorted(os.walk(dir, followlinks=True)):
        for fname in fnames:
            if is_image_file(fname):
                path = os.path.join(root, fname)
                images.append(path)
    return images[:min(max_dataset_size, len(images))]


class ImageFolder(data.Dataset):
    def __init__(self, root, transform=None):
        imgs = make_dataset(root)
        if len(imgs) == 0:
            raise(RuntimeError("Found 0 images in: " + root + "\n"
                               "Supported image extensions are: " + ",".join(IMG_EXTENSIONS)))
        self.root = root
        self.imgs = imgs
        self.transform = transform
        self.bin_hf = h5py.File('/content/drive/MyDrive/EECS442Group/bin_data_64.h5', 'r')
        self.L_hf = h5py.File('/content/drive/MyDrive/EECS442Group/L_data.h5', 'r')
        self.AB_hf = h5py.File('/content/drive/MyDrive/EECS442Group/AB_data.h5', 'r')

    def __getitem__(self, index):
        # get the image path
        path = self.imgs[index]
        #path = "cocostuff-2017\\" + os.path.basename(os.path.normpath(path))
        path = "celeba\\" + os.path.basename(os.path.normpath(path))

        # retrieve the L channel and binned image
        L_img = self.L_hf.get(path)
        L_img = np.array(L_img)
        bin_img = self.bin_hf.get(path)
        bin_img = np.array(bin_img)

        # apply transforms and return
        L_img = self.transform(L_img)
        bin_img = self.transform(bin_img)
        return L_img, bin_img

    def __len__(self):
        return len(self.imgs)


Create our model

In [ ]:
# helper function for Zhang et al. 2016 architecture structure - 2 CONV layers
def make_block_2conv(layers, in_channels, out_channels):
    layers.append(nn.Conv2d(in_channels, out_channels, 3, 1, 1))
    layers.append(nn.ReLU(inplace=True))
    layers.append(nn.Conv2d(out_channels, out_channels, 3, 2, 1))
    layers.append(nn.ReLU(inplace=True))
    layers.append(nn.BatchNorm2d(out_channels))

# helper function for Zhang et al. 2016 architecture structure - 3 CONV layers
def make_block_3conv(layers, in_channels, out_channels, kernel_size=3, stride=1, padding=1, dilation=1):
    layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, 1, padding, dilation=dilation))
    layers.append(nn.ReLU(inplace=True))
    layers.append(nn.Conv2d(out_channels, out_channels, kernel_size, 1, padding, dilation=dilation))
    layers.append(nn.ReLU(inplace=True))
    layers.append(nn.Conv2d(out_channels, out_channels, kernel_size, stride, padding, dilation=dilation))
    layers.append(nn.ReLU(inplace=True))
    layers.append(nn.BatchNorm2d(out_channels))

class Colorizer(nn.Module):
    def __init__(self):
        super(Colorizer, self).__init__()

        self.layers = []

        make_block_2conv(self.layers, 1, 32)
        make_block_2conv(self.layers, 32, 64)
        make_block_3conv(self.layers, 64, 128, 3, stride=2)
        make_block_3conv(self.layers, 128, 256, 3)
        make_block_3conv(self.layers, 256, 256, 3, padding=2, dilation=2)
        make_block_3conv(self.layers, 256, 256, 3)

        self.layers.append(nn.ConvTranspose2d(256, 128, 4, 2, 1))
        self.layers.append(nn.ReLU(inplace=True))
        self.layers.append(nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True))
        self.layers.append(nn.ReLU(inplace=True))
        self.layers.append(nn.ConvTranspose2d(128, 128, 4, 2, 1))
        self.layers.append(nn.ReLU(inplace=True))
        self.layers.append(nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True))
        self.layers.append(nn.ReLU(inplace=True))
        self.layers.append(nn.ConvTranspose2d(128, 64, 4, 2, 1))
        self.layers.append(nn.ReLU(inplace=True))

        self.layers.append(nn.Conv2d(64, 313, 1, 1, 0))

        self.model = nn.Sequential(*self.layers)

    def forward(self, x):
        x = self.model(x)

        return x

Data Loader

In [ ]:
class Bin_Converter():
    def __init__(self):
        # https://github.com/richzhang/colorization
        self.palette = np.load("/content/drive/MyDrive/EECS442Group/richzhang_palette/pts_in_hull.npy")

        # add our own counts and divide to get prior probabilities
        prior = np.load("/content/drive/MyDrive/EECS442Group/celeb_counts.npy")
        prior[prior == 0] = 1
        prior = prior / np.sum(prior)

        # self.weights = 0.8 + prior 

        # Weights were taken from Time0o on github
        # https://github.com/Time0o/pytorch-colorful-colorization/tree/9cbbc9fb7518bd92c441e36e45466cfd663fa9db
        lambda_ = 0.5
        #Apply function for weights
        uniform = np.zeros_like(prior)
        uniform[prior > 0] = 1 / (prior > 0).sum()

        self.weights = 1 / ((1 - lambda_) * prior + lambda_ * uniform)
        self.weights /= np.sum(prior * self.weights)

    # image is a numpy array CxHxW
    def convert_bin(self, image):
        # get the l2 difference between binned AB values and bins
        bin_image = np.zeros((image.shape[1], image.shape[2]))

        for x in range(image.shape[1]):
            for y in range(image.shape[2]):
                bin_dists = np.linalg.norm(np.abs(self.palette - image[:, x, y]), axis=1)
                bin_image[x, y] = np.argmin(bin_dists)

        return bin_image

    def convert_AB(self, image):
        AB_image = np.zeros((2, image.shape[0], image.shape[1]))
        for x in range(image.shape[0]):
            for y in range(image.shape[1]):
                AB_image[0, x, y] = self.palette[image[x, y].astype(np.int64), 0]
                AB_image[1, x, y] = self.palette[image[x, y].astype(np.int64), 1]
        return AB_image

if __name__ == "__main__":
    
    test = Bin_Converter()


In [ ]:
#mount on google drive
!ls /content/drive/MyDrive/EECS442Group

bin_hf = h5py.File('/content/drive/MyDrive/EECS442Group/bin_data_64.h5', 'r')
L_hf = h5py.File('/content/drive/MyDrive/EECS442Group/L_data.h5', 'r')
AB_hf = h5py.File('/content/drive/MyDrive/EECS442Group/AB_data.h5', 'r')
converter = Bin_Converter()

#images = make_dataset("/content/drive/MyDrive/EECS442Group/cocostuff-2017")
images = make_dataset("/content/drive/MyDrive/EECS442Group/celeba")

# Determine size of dataset
counter = 1
for image_path in images:
    counter += 1
print("number of images:", counter)

 AB_data.h5	    colorizer_alternate_dataset.ipynb   L_data.h5
 AB_flower.h5	    colorizer_annika.ipynb	        L_flower.h5
 bin_data_64.h5     colorizer_flower.ipynb	        original.jpg
 bin_flower_64.h5   colorizer_flower_none.ipynb         output_images
 bin_images	    colorizer.ipynb		       'Project Proposal.gdoc'
 celeba		    dataset.py			        project_proposal.pdf
 celeb_counts.npy  'Final Presentation.gslides'        'Project Report.gdoc'
 checkpoints	    flower_counts.npy		        richzhang_palette
 cocostuff-2017     flowers			        val_data
number of images: 6001


Checkpoint stuff!

In [ ]:
# Initialize Model
net = Colorizer().to(device)
print("Model Summary:")
summary(net, (1,64,64))

def save_checkpoint(i,net,loss):
  EPOCH = i
  PATH = "model"+str(i)+".pt"
  LOSS = loss

  torch.save({
              'epoch': EPOCH,
              'model_state_dict': net.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': LOSS,
              }, PATH)

#USE THIS IN THE EVENT OF A CRASH TO RETRIEVE MODEL
"""
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
"""

Model Summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             320
              ReLU-2           [-1, 32, 64, 64]               0
            Conv2d-3           [-1, 32, 32, 32]           9,248
              ReLU-4           [-1, 32, 32, 32]               0
       BatchNorm2d-5           [-1, 32, 32, 32]              64
            Conv2d-6           [-1, 64, 32, 32]          18,496
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 16, 16]          36,928
              ReLU-9           [-1, 64, 16, 16]               0
      BatchNorm2d-10           [-1, 64, 16, 16]             128
           Conv2d-11          [-1, 128, 16, 16]          73,856
             ReLU-12          [-1, 128, 16, 16]               0
           Conv2d-13          [-1, 128, 16, 16]         147,584
             ReLU-14    

"\ncheckpoint = torch.load(PATH)\nmodel.load_state_dict(checkpoint['model_state_dict'])\noptimizer.load_state_dict(checkpoint['optimizer_state_dict'])\nepoch = checkpoint['epoch']\nloss = checkpoint['loss']\n"

Training

In [ ]:
def init_arguments():
    parser = {}
    parser['data_path'] = "/content/drive/MyDrive/EECS442Group/celeba"
    parser['image_size'] = 128
    parser['num_epochs'] = 100
    parser['batch_size'] = 8
    parser['lr'] = 2e-4
    parser['beta1'] = 0.5
    parser['beta2'] =0.999
    parser['eps'] = 1e-08
    parser['weight_decay'] = 1e-4
    parser['print_loss_freq'] = 10
    parser['checkpoint_freq'] = 10
    parser['experiment_idx'] = "5"
    return parser


def train(model, trainloader, valloader, num_epoch, optimizer, criterion, args, device): # Train the model
    print("Start training...")
    trn_loss_hist = []
    val_loss_hist = []
    # EXPERIMENTAL
    mse_crit = nn.MSELoss()
    # https://github.com/richzhang/colorization
    priors = torch.tensor(np.load("/content/drive/MyDrive/EECS442Group/richzhang_palette/prior_probs.npy")).to(device)

    for i in range(num_epoch):
        model.train()
        running_loss = []
        for idx, data in enumerate(trainloader):
            optimizer.zero_grad()
            L_img = data[0].to(device)
            bin_img = data[1].to(device).long()
            pred = model(L_img)
            # pred is NxCxHxW and C is 313
            pred = pred.permute(0, 2, 3, 1).flatten(1, 2)
            pred = pred.flatten(0, 1)
            bin_img = bin_img.permute(0, 2, 3, 1).flatten(1, 2)
            bin_img = bin_img.flatten(0, 1).squeeze()
            # make into one long N*H*WxC vector for both and compare
            # loss = criterion(pred, bin_img)
            # EXPERIMENTAL
            ce_loss = criterion(pred, bin_img)
            # pred = torch.argmax(pred, axis=1)
            # counts = torch.bincount(pred, minlength=313)
            # counts = counts / torch.sum(counts)
            counts = torch.sum(pred, axis=0, dtype=float)
            counts /= torch.sum(counts)
            mse_loss = mse_crit(counts, priors)
            loss = 0.6*ce_loss + 0.4 * mse_loss
            # END EXPERIMENTAL
            running_loss.append(loss.item())
            loss.backward()
            optimizer.step()            

        model.eval()
        running_val_loss = []
        with torch.no_grad():
            for idx, data in enumerate(valloader):
                L_img = data[0].to(device)
                bin_img = data[1].to(device).long()
                pred = model(L_img)
                pred = pred.permute(0, 2, 3, 1).flatten(1, 2)
                pred = pred.flatten(0, 1)
                bin_img = bin_img.permute(0, 2, 3, 1).flatten(1, 2)
                bin_img = bin_img.flatten(0, 1).squeeze()
                loss = criterion(pred, bin_img)
                running_val_loss.append(loss.item())

        train_loss = np.mean(running_loss)
        val_loss = np.mean(running_val_loss)
        print("\nEpoch {} train loss: {} val loss: {}".format(i + 1, train_loss, val_loss))
        trn_loss_hist.append(train_loss)
        val_loss_hist.append(val_loss)

        #SAVE CHECKPOINT every 10 iterations to EECS442Group in gdrive
        """
        if (i % 10) == 0:
          save_checkpoint(i, model, np.mean(running_loss))
        """
        if (i+1) % args['checkpoint_freq'] == 0:
          # CHANGE
          experiment_number = '5'
          torch.save({'epoch': i,
                      'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                      'loss': train_loss,
                      }, '/content/drive/MyDrive/EECS442Group/checkpoints/model_'+str(i+1)+'_'+experiment_number+'.pt')
        #model_name should be model_epoch_idx.pt
        
    return trn_loss_hist, val_loss_hist


def init_transform(args):
    # if we flip, then need to flip both
    transform = transforms.Compose([
            transforms.ToTensor()
        ])
    return transform


def evaluate(model, loader):
  model.eval() # Set the model to evaluation mode
  correct = 0
  with torch.no_grad(): # Do not calculate gradient to speed up computation
    for batch, label in tqdm(loader):
      batch = batch.to(device)
      label = label.to(device)
      pred = model(batch)
      correct += (torch.argmax(pred,dim=1)==label).sum().item()
    acc = correct/len(loader.dataset)
    print("\n Evaluation accuracy: {}".format(acc))
    return acc


In [ ]:
# Grab hyperparameters from command line
args = init_arguments()
transform = init_transform(args)

# Load Dataset
bin_converter = Bin_Converter()
dataset = ImageFolder(args['data_path'], transform)
train_set = Subset(dataset, range(4800))
val_set = Subset(dataset, range(4800, 5400))
test_set = Subset(dataset, range(5400, 6000))
trainloader = torch.utils.data.DataLoader(train_set, batch_size=args['batch_size'])
valloader = torch.utils.data.DataLoader(val_set, batch_size=args['batch_size'])

# Define loss function, and optimizer
class_rebalancing = torch.tensor(bin_converter.weights, dtype=torch.float).to(device)

criterion = torch.nn.CrossEntropyLoss(weight=class_rebalancing)
optimizer = torch.optim.Adam(net.parameters(), lr=args["lr"], weight_decay=args["weight_decay"])

# Train
trn_loss_hist, val_loss_hist = train(net, trainloader, valloader, args['num_epochs'], optimizer, criterion, args, device)

Start training...

Epoch 1 train loss: 1.9641752462547784 val loss: 3.2655819606781007

Epoch 2 train loss: 1.948994374561568 val loss: 3.3091069793701173

Epoch 3 train loss: 1.9355079323297901 val loss: 3.2894660313924153

Epoch 4 train loss: 1.9215926583598661 val loss: 3.293380257288615

Epoch 5 train loss: 1.9094923856790826 val loss: 3.3359070682525633

Epoch 6 train loss: 1.8939193423676277 val loss: 3.3009297180175783

Epoch 7 train loss: 1.8795985195183835 val loss: 3.3092792733510334

Epoch 8 train loss: 1.8585095669015508 val loss: 3.3508730570475262

Epoch 9 train loss: 1.8462129141829398 val loss: 3.3678043333689374

Epoch 10 train loss: 1.8310133034213816 val loss: 3.3818511613210043

Epoch 11 train loss: 1.809673013570232 val loss: 3.4138476753234865

Epoch 12 train loss: 1.7902212252609546 val loss: 3.5003252442677817

Epoch 13 train loss: 1.7747073342081876 val loss: 3.4403092861175537

Epoch 14 train loss: 1.7550027508935546 val loss: 3.4706359640757243

Epoch 15 trai

Evaluate Performance

In [ ]:
# Helper function for opening test image
def open_img_error_check(image_path):
  try:
    return Image.open(image_path).convert('RGB')
  except:
    return "error"

# set epoch of choice
load_epoch = 100

bin_converter = Bin_Converter()
model = Colorizer().to(device)
# CHANGE
experiment_number = '5'
checkpoint = torch.load('/content/drive/MyDrive/EECS442Group/checkpoints/model_'+str(load_epoch)+"_"+ experiment_number + '.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

dataset_pathway = "/content/drive/MyDrive/EECS442Group/celeba"
save_img = True

ssim_scores = []
psnr_scores = []
counter = 5400
with torch.no_grad():
    for test_image in test_set:
        img_idx = str(counter).zfill(6)
        image_path = str(dataset_pathway) + "/" + str(img_idx) + ".jpg"

        img = open_img_error_check(image_path)

        # Only opens existing images since naming mechanism isn't sequential in celeba gdrive
        if img == "error":
          counter += 1
        else:
          img = img.resize((64, 64), Image.BICUBIC)
          img = np.array(img)
          original_img = img
          img = color.rgb2lab(img).astype(np.float32)

          dict_key = "celeba\\" + str(img_idx) + ".jpg"
          L_img = np.array(L_hf.get(dict_key))
          AB_img = AB_hf.get(dict_key)

          input = transform(L_img).to(device)
          input = torch.unsqueeze(input, 0)
          pred = model(input)
          pred = np.squeeze(pred.cpu().numpy())

          pred_AB = []
          # change 1-5
          num_max_bins = 2
          for i in range(num_max_bins):
              max = np.argmax(pred, axis=0)
              pred[max] = -100000
              pred_AB.append(bin_converter.convert_AB(max))

          test = np.dstack((img[:, :, 0], pred_AB[0].transpose(1,2,0)))
          test = (255 * np.clip(color.lab2rgb(test), 0, 1)).astype(np.uint8)

          pred_AB = np.mean(np.array(pred_AB), axis=0)
          pred_AB = pred_AB.transpose(1, 2, 0)

          output = np.dstack((img[:, :, 0], pred_AB))
          colorized_img = output
          output = (255 * np.clip(color.lab2rgb(output), 0, 1)).astype(np.uint8)

          (ssim_score, diff) = compare_ssim(original_img, output, full=True, multichannel=True)
          psnr_score = peak_signal_noise_ratio(original_img, output)
          ssim_scores.append(ssim_score)
          psnr_scores.append(psnr_score)
          if save_img:
            plt.imsave("/content/drive/MyDrive/EECS442Group/output_images/L_img.jpg", L_img, cmap="gray")
            save_img = False
            filename = "/content/drive/MyDrive/EECS442Group/output_images/"+str(img_idx)+str(counter)+"_"+experiment_number+'.jpg'
            plt.imsave(filename, output)

          counter += 1
          print(counter)
    print(np.mean(ssim_scores), np.mean(psnr_scores))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.


5401
5402
5403
5404


/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 88 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 29 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 10 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


5405
5406
5407
5408
5409
5410
5411
5412


/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 5 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 34 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 33 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


...

5901
5902
5903
5904
5905
5906
5907
5908
5909
5910
5911
5912
5913
5914
5915
5916
5917
5918
5919
5920


/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 20 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


5921
5922
5923
5924
5925
5926
5927
5928
5929
5930
5931
5932
5933
5934
5935
5936
5937
5938
5939
5940
5941
5942
5943
5944
5945
5946
5947
5948
5949
5950
5951
5952
5953
5954
5955
5956
5957
5958
5959
5960
5961
5962
5963
5964
5965
5966
5967
5968
5969
5970
5971
5972
5973
5974
5975
5976
5977
5978
5979
5980
5981
5982
5983
5984
5985
5986
5987
5988


/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 51 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


5989
5990
5991
5992
5993
5994
5995
5996
5997
5998
5999
6000
0.9032888626089765 26.12301752596126
